In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/상장raw.csv')

In [4]:
df['Year']= df['회계년도'].replace('/','').astype(str).str[:4].astype(int)

In [14]:
df[(df['거래소코드'] == df['거래소코드'].shift(-1)) & (df['Year'].diff(-1) != -1) &(df['감사의견코드(IFRS)'] ==1)]

,회사명,거래소코드,회계년도,통계청 한국표준산업분류 코드 10차(대분류),통계청 한국표준산업분류 10차(대분류),부채비율(IFRS연결),당좌비율(IFRS),유동비율(IFRS),이자보상배율(이자비용)(IFRS),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),...,자본(*)(IFRS연결)(천원),연구개발비(IFRS연결)(천원),* (정상)영업손익(보고서기재)(IFRS연결)(천원),PER(최고)(IFRS),PER(최저)(IFRS),PBR(최고)(IFRS),PBR(최저)(IFRS),자본금(*)(IFRS연결)(천원),감사의견코드(IFRS),Year
10156,(주)이큐셀,160600,2019/12,29,제조업,382.87,63.03,71.91,-18.41,-18682596.0,...,16974514.0,2232346.0,-29898119.0,0.0,0.0,11.26,1.03,15068102.0,1.0,2019


In [11]:
df['감사의견코드(IFRS)']=df['감사의견코드(IFRS)'].map({
    'AG': 1,
    'DS':1,
    'DI':1,
    'DU':1,
    'QA':0,
    'QC':0,
    'QG':0,
    'QQ':0,
    'QS':1,
    'QU':0,
    'UQ':0,
    'ZZ':1
})

In [12]:
a = df[df['Year'] != 2012]

In [13]:
a[a['회사명'].str.contains('라온텍')]

,회사명,거래소코드,회계년도,통계청 한국표준산업분류 코드 10차(대분류),통계청 한국표준산업분류 10차(대분류),부채비율(IFRS연결),당좌비율(IFRS),유동비율(IFRS),이자보상배율(이자비용)(IFRS),영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원),...,자본(*)(IFRS연결)(천원),연구개발비(IFRS연결)(천원),* (정상)영업손익(보고서기재)(IFRS연결)(천원),PER(최고)(IFRS),PER(최저)(IFRS),PBR(최고)(IFRS),PBR(최저)(IFRS),자본금(*)(IFRS연결)(천원),감사의견코드(IFRS),Year
2519,(주)라온텍,418420,2013/12,26,제조업,49.47,221.74,322.4,-51.63,-19302.0,...,6452643.0,4265.0,-2468000.0,0.0,0.0,0.0,0.0,2175000.0,NaN,2013
2520,(주)라온텍,418420,2021/12,26,제조업,NaN,NaN,NaN,NaN,591946.0,...,5791910.0,2701154.0,-1564126.0,NaN,NaN,NaN,NaN,2496175.0,0.0,2021
2521,(주)라온텍,418420,2022/12,26,제조업,68.26,372.51,483.3,8.65,404090.0,...,7482903.0,4234868.0,322445.0,0.0,0.0,0.0,0.0,2496175.0,0.0,2022


In [16]:
df_외감 = pd.read_csv('../data/외감raw.csv', encoding='euc-kr')

In [22]:
df_외감['감사의견코드']=df_외감['감사의견코드'].map({
    'AG': 1,
    'DS':1,
    'DI':1,
    'DU':1,
    'QA':0,
    'QC':0,
    'QG':0,
    'QQ':0,
    'QS':1,
    'QU':0,
    'UQ':0,
    'ZZ':1
})

In [18]:
# '거래소코드'로 그룹화하여 처리
grouped = df_외감.groupby('거래소코드')

df_외감['새로운 인덱스'] = grouped.cumcount() + 1

# 처리할 새로운 DataFrame을 저장할 리스트 초기화
result_data = []

for group_key, group_df_외감 in grouped:
    # '감사의견코드' 컬럼 값이 1인 행들 중 첫 번째 행의 인덱스를 구함
    first_idx = group_df_외감[group_df_외감['감사의견코드'] == 1].index[0]
    
    if group_df_외감.loc[first_idx, '감사의견코드'] == 1:
        # 그룹화된 첫 번째 행이면 해당 '거래소코드'를 가진 행들은 삭제
        group_df_외감.drop(index=group_df_외감.index, inplace=True)
    else:
        # 그룹화된 첫 번째 행이 아니면 바로 위의 행의 't-1 감사의견코드' 컬럼에 1 입력
        group_df_외감.loc[first_idx - 1, 't-1 감사의견코드'] = 1
        result_data.append(group_df_외감)

# 처리 결과를 하나의 DataFrame으로 병합
result_df_외감 = pd.concat(result_data)

result_df_외감


IndexError: index 0 is out of bounds for axis 0 with size 0

In [23]:
import pandas as pd

# 거래소코드로 그룹화하여 처리하는 함수
def process_group(group):
    # 빈 그룹인 경우 그대로 반환
    if group.empty:
        return group
    
    # 거래소코드 그룹별로 처리
    if 1 in group['감사의견코드'].values:
        # 1이 처음으로 나오는 인덱스 찾기
        first_index = group.index[group['감사의견코드'] == 1][0]
        
        # 처음으로 1이 나오는 행의 아래 행들 삭제
        group = group.loc[:first_index]
    else:
        # 1이 없는 경우 그룹 그대로 유지
        pass
    
    return group

# '거래소코드'로 그룹화하여 함수를 적용하고 다시 합치기
processed_df = df_외감.groupby('거래소코드').apply(process_group)

processed_df


회사명   거래소코드     회계년도  통계청 한국표준산업분류 코드 10차(대분류)    부채비율  \
거래소코드                                                                         
10000  269007   (주)포스코건설   10000  2012/12                        41  173.49   
       269008   (주)포스코건설   10000  2013/12                        41  130.89   
       269009   (주)포스코건설   10000  2014/12                        41  120.92   
       269010   (주)포스코건설   10000  2015/12                        41  100.59   
       269011   (주)포스코건설   10000  2016/12                        41  145.02   
...                  ...     ...      ...                       ...     ...   
107274 215212  주식회사이안이앤디  107274  2022/12                        68    0.00   
107276 216611    주식회사일그램  107276  2021/12                        71   99.28   
       216612    주식회사일그램  107276  2022/12                        71  145.67   
107278 32679   (주)두산종합건재  107278  2021/12                        46  273.78   
       32680   (주)두산종합건재  107278  2022/12                        46  909.68   

                  당좌비율     유동비율  이자보상배율(이자비용)  \
거래소코드                                           
10000  269007   124.35   149.99         14.79   
       269008   120.59   146.23         13.84   
       269009   125.34   153.62          6.33   
       269010   145.66   164.20          7.33   
       269011   109.23   133.86         -8.08   
...                ...      ...           ...   
107274 215212    11.04    79.11         -0.71   
107276 216611  1838.90  1838.90         25.74   
       216612   448.46   448.46         13.22   
107278 32679    376.36   394.26          1.13   
       32680    284.24   291.35         -6.44   

               영업활동으로 인한 현금흐름(간접법)(*)(IFRS연결)(천원)  이자비용(IFRS연결)(천원)  ...  \
거래소코드                                                                ...   
10000  269007                         123580817.0        45854000.0  ...   
       269008                         383458439.0        44520000.0  ...   
       269009                         -81029724.0        55648000.0  ...   
       269010                         645297841.0        61387000.0  ...   
       269011                        -616838633.0        45847000.0  ...   
...                                           ...               ...  ...   
107274 215212                         -20227789.0         1314616.0  ...   
107276 216611                           3232717.0          108993.0  ...   
       216612                           3770208.0          315760.0  ...   
107278 32679                           -2107626.0          241431.0  ...   
       32680                             154239.0          328282.0  ...   

               투자활동으로 인한 현금흐름(*)(IFRS연결)(천원)  재무활동으로 인한 현금흐름(*)(IFRS연결)(천원)  \
거래소코드                                                                         
10000  269007                   -331194515.0                    121474510.0   
       269008                   -150352822.0                   -163581203.0   
       269009                   -527581873.0                    -27739975.0   
       269010                      5879907.0                    391500030.0   
       269011                   -244628460.0                    146163668.0   
...                                      ...                            ...   
107274 215212                      -523459.0                     20739181.0   
107276 216611                     -9688828.0                      6879900.0   
       216612                    -10180053.0                      5830000.0   
107278 32679                       -825371.0                      2865000.0   
       32680                       -192254.0                        79100.0   

               자산(*)(IFRS연결)(천원)  부채(*)(IFRS연결)(천원)  자본(*)(IFRS연결)(천원)  \
거래소코드                                                                    
10000  269007       9.200763e+09       6.371979e+09       2.828785e+09   
       269008       8.641621e+09       5.695142e+09       2.946479e+09   
       269009       8.366265e+09       5.3

In [24]:
# processed_df.to_csv('test.csv', encoding='euc-kr')

In [26]:
processed_df['감사의견코드'].value_counts()

0.0    208286
1.0     15598
Name: 감사의견코드, dtype: int64

In [29]:
# 처리할 새로운 DataFrame을 저장할 리스트 초기화
result_data = []

for group_key, group_processed_df in grouped:
    # 'target' 컬럼 값이 1인 행들 중 첫 번째 행의 인덱스를 구함
    first_idx = group_processed_df[group_processed_df['감사의견코드'] == 1].index[0]
    
    if group_processed_df.loc[first_idx, '감사의견코드'] == 1:
        # 그룹화된 첫 번째 행이면 해당 '거래소코드'를 가진 행들은 삭제
        group_processed_df.drop(index=group_processed_df.index, inplace=True)
    else:
        # 그룹화된 첫 번째 행이 아니면 바로 위의 행의 't-1 감사의견코드' 컬럼에 1 입력
        group_processed_df.loc[first_idx - 1, 't-1 감사의견코드'] = 1
        result_data.append(group_processed_df)

# 처리 결과를 하나의 DataFrame으로 병합
result_processed_df = pd.concat(result_data)

result_processed_df


IndexError: index 0 is out of bounds for axis 0 with size 0

In [30]:
result_data = []

for group_key, group_processed_df in grouped:
    # 그룹화된 데이터프레임이 비어있는 경우, 처리를 건너뛰고 다음 그룹으로 넘어감
    if group_processed_df.empty:
        continue
    
    # 'target' 컬럼 값이 1인 행들 중 첫 번째 행의 인덱스를 구함
    first_idx = group_processed_df[group_processed_df['감사의견코드'] == 1].index[0]
    
    if group_processed_df.loc[first_idx, '감사의견코드'] == 1:
        # 그룹화된 첫 번째 행이면 해당 '거래소코드'를 가진 행들은 삭제
        group_processed_df.drop(index=group_processed_df.index, inplace=True)
    else:
        # 그룹화된 첫 번째 행이 아니면 바로 위의 행의 't-1 감사의견코드' 컬럼에 1 입력
        group_processed_df.loc[first_idx - 1, 't-1 감사의견코드'] = 1
        result_data.append(group_processed_df)

# 처리 결과를 하나의 DataFrame으로 병합
result_processed_df = pd.concat(result_data)

result_processed_df

IndexError: index 0 is out of bounds for axis 0 with size 0